In [ ]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import scipy.stats as st




def Exercise_5_2():
    K = 10
    M = 7
    z_K = []
    np.random.seed(2)
    confidence = 0.95
    alpha = 1-confidence
    z = st.norm.ppf(1-alpha/2)
    x = 94.26497124491036 # obtained from SAA when calculating lower bound

    
    for m in range(M):
        q_O = 1.2
        q_B = np.random.uniform(1.5,2.2,K)
        d = np.random.uniform(80,120,K)
        m = gp.Model("5_2")
        y_B = m.addVars([i for i in range(K)], vtype = GRB.CONTINUOUS, name = "the amount bought in the local market ")
        y_O = m.addVars([i for i in range(K)], vtype = GRB.CONTINUOUS, name = "the oversupply")
        
        m.setObjective(x + (1/K)*sum(q_B[k]*y_B[k]+q_O*y_O[k] for k in range(K)),GRB.MINIMIZE)
        
        
        for k in range(K):
            m.addConstr(y_B[k]-y_O[k]+x == d[k])
  
        
        m.optimize()
        
        z_K.append(m.getObjective().getValue())
        print('y_B:')
        for i in range(K):
            print(y_B[i].x)

        print('y_O:')
        for i in range(K):
            print(y_O[i].x)
    
    
    
    U_MK = sum(z_K)/M
    
    
    #standard deviation
    theta = (1/(M-1)*sum((z_K[i]-U_MK)**2 for i in range(M)))**(1/2)
    
    cinterval = [U_MK-theta*z/(M**(1/2)),U_MK+theta*z/(M**(1/2))]
        
    print('z_K',z_K)
    print('U_MK',U_MK)
    print('standard deviation',theta)
    print('confidence interval', cinterval)